<a href="https://colab.research.google.com/github/isabelleBarbier/isabelleBarbier/blob/main/Copie_de_Breast_cancer_06_11_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import random
import sys
import cv2
import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Import des données

In [1]:
!git clone https://github.com/MachineLearnia/breast_cancer_public_data.git

Cloning into 'breast_cancer_public_data'...
remote: Enumerating objects: 4203, done.
remote: Counting objects: 100% (805/805), done.
remote: Compressing objects: 100% (805/805), done.
remote: Total 4203 (delta 0), reused 805 (delta 0), pack-reused 3398 (from 2)
Receiving objects: 100% (4203/4203), 90.60 MiB | 21.52 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
!ls

breast_cancer_public_data  sample_data


In [6]:
os.chdir("breast_cancer_public_data")

In [7]:
!ls

data  data_2  README.md


In [8]:
folder_path = "./data_2"

In [9]:
# Train data preparation
classes = ["Negative", "Cancer"]
dataset = []
for class_label in classes:
    class_path = os.path.join(folder_path, class_label)
    label_index = classes.index(class_label)
    for img_file in tqdm.tqdm(os.listdir(class_path)):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        dataset.append([img, label_index])


100%|██████████| 410/410 [00:00<00:00, 1753.49it/s]


In [10]:
len(dataset)

820

# Normalisation

In [ ]:
X_images = []
y_images = []
for img, label in dataset:
    X_images.append(img)
    y_images.append(label)


X = np.array(X_images)
y_images = np.array(y_images)


# Découpe en un jeu d'entrainement et un jeu de Test

# Entrainement du CNN :

In [ ]:
model = Sequential()

model.add(....)

model.summary()

In [ ]:
model.compile(...)

In [ ]:
# Training the model
history = model.fit(....)

# Visualisation des résultats